# Descriptive analytics AUs data

1. The folder contains 128 files. I don't really understand how the names of the files are structured and what they mean. The last part corresponds to the different time points (3, 4, 5, 6, 7), correct? I first thought that the first number would correspond to an id of a participant, but there are only 128 files, and there are 281 unique IDs. Is it also correct that the ID starts numbering at 5? 

I see 414 files; I downloaded the same file I sent you just to be sure. The participant number starts from 5, that's correct. The file is named participant ID - time point, e.g. 5-03-04-05-06 it means that this file contains AU extracted from video number 5 (participant ID), time points 3, 4, 5, 6 (this is an entire blood donation, we have continuous video).

2. How can I link the IDs in the donor info2 csv to the files with the AUs? 

You can link by participant ID (e.g. 5, 6, 311).


3. Would you recommend me making one big file and putting everything together to run my models? 

You can, it depends on how you want to run the analysis (but this file is going to be big and takes time even to open it).

Just another tip: before you proceed with data analysis, there is a column named 'confidence' in each file. Make sure those data points you are going to use have a high confidence score (above 80-90).

In [20]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import zipfile 
import io

"""
In the Excel file 20221101_donor_info2 you will find the participant's ID which will match all RGB files and then column 'Gender' (1 - male, 2 - female).
RGB Excel files are preprocessed using OpenFace, probably you are interested in all the fields that have AU and end with 'r', e.g. AU45_r - 
these columns show the intensity of each extracted AU)

There are 414 files in the folder, all of them with 714 columns

I think to prepare the data I should make one big file of everything together (I need to know how to connect the AUs to the ID though)
Then I can drop a lot of the variables, as I'm particularly interested in the field that have AU and end with r. 
"""

zip_file_path = "/Users/dionnespaltman/Desktop/downloading/full donation data.zip"

def extract_and_open_csv(zip_file_path, csv_file_name):
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        with zip_ref.open(csv_file_name) as file:
            df = pd.read_csv(io.BytesIO(file.read()))
    return df

# The first file in the zip 
csv_file_name_100 = "full donation data/100_04,05,06.csv"
df_100 = extract_and_open_csv(zip_file_path, csv_file_name_100)
display(df_100)

frame   face_id   timestamp   confidence   success   gaze_0_x  \
0          1         0        0.00         0.03         1   0.218845   
1          2         0        0.04         0.03         0   0.192400   
2          3         0        0.08         0.03         0   0.312787   
3          4         0        0.12         0.03         0   0.248890   
4          5         0        0.16         0.03         0   0.382508   
...      ...       ...         ...          ...       ...        ...   
24645  24646         0      985.80         0.98         1   0.000776   
24646  24647         0      985.84         0.98         1   0.025698   
24647  24648         0      985.88         0.98         1   0.027142   
24648  24649         0      985.92         0.98         1   0.049283   
24649  24650         0      985.96         0.98         1   0.029234   

        gaze_0_y   gaze_0_z   gaze_1_x   gaze_1_y  ...   AU12_c   AU14_c  \
0       0.232655  -0.947617   0.109746   0.146850  ...      0.0      0.0   
1       0.252572  -0.948256   0.088528   0.202929  ...      0.0      0.0   
2       0.316749  -0.895452   0.104171   0.215143  ...      0.0      0.0   
3       0.315923  -0.915558   0.093714   0.262643  ...      0.0      0.0   
4       0.366388  -0.848203   0.068176   0.215664  ...      0.0      0.0   
...          ...        ...        ...        ...  ...      ...      ...   
24645   0.202148  -0.979355  -0.131025   0.277231  ...      1.0      0.0   
24646   0.196356  -0.980196  -0.138288   0.283798  ...      1.0      0.0   
24647   0.216634  -0.975875  -0.142773   0.290942  ...      1.0      0.0   
24648   0.227999  -0.972413  -0.130225   0.265479  ...      1.0      0.0   
24649   0.211907  -0.976852  -0.117023   0.268408  ...      1.0      0.0   

        AU15_c   AU17_c   AU20_c   AU23_c   AU25_c   AU26_c   AU28_c   AU45_c  
0          0.0      1.0      1.0      1.0      1.0      1.0      0.0      0.0  
1          0.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0  
2          0.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0  
3          0.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0  
4          0.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0  
...        ...      ...      ...      ...      ...      ...      ...      ...  
24645      0.0      0.0      1.0      0.0      1.0      1.0      0.0      0.0  
24646      0.0      0.0      1.0      0.0      1.0      1.0      0.0      0.0  
24647      0.0      0.0      1.0      1.0      1.0      1.0      0.0      0.0  
24648      0.0      0.0      1.0      0.0      1.0      1.0      0.0      0.0  
24649      0.0      0.0      1.0      1.0      1.0      1.0      0.0      0.0  

[24650 rows x 714 columns]

In [2]:
print(df_100.shape)

(24650, 714)


In [3]:
columns_to_drop = [col for col in df_100.columns if any(x in col for x in ['p', 'Z', 'Y', 'y', 'x', 'gaze', 'eye', 'X', '_c'])]
df_100_dropped = df_100.drop(columns=columns_to_drop)

print(df_100_dropped.shape)

(24650, 21)


In [4]:
#print(df_100.columns)
df_100 = df_100.rename(columns={' face_id': 'face_id'})
print(f"Number of unique values in the face ID column is {df_100.face_id.nunique()}.")

Number of unique values in the face ID column is 1.


In [5]:
n_rows, n_columns = df_100.shape
print("Number of rows:", n_rows)
print("Number of columns:", n_columns)

Number of rows: 24650
Number of columns: 714


In [6]:
column_names = df_100.columns.tolist()
print(column_names)
au_column_names = [col for col in column_names if 'AU' in col]
au_r_column_names = [col for col in au_column_names if 'r' in col]
#print(au_column_names)
print(au_r_column_names)
print(len(au_r_column_names))

['frame', 'face_id', ' timestamp', ' confidence', ' success', ' gaze_0_x', ' gaze_0_y', ' gaze_0_z', ' gaze_1_x', ' gaze_1_y', ' gaze_1_z', ' gaze_angle_x', ' gaze_angle_y', ' eye_lmk_x_0', ' eye_lmk_x_1', ' eye_lmk_x_2', ' eye_lmk_x_3', ' eye_lmk_x_4', ' eye_lmk_x_5', ' eye_lmk_x_6', ' eye_lmk_x_7', ' eye_lmk_x_8', ' eye_lmk_x_9', ' eye_lmk_x_10', ' eye_lmk_x_11', ' eye_lmk_x_12', ' eye_lmk_x_13', ' eye_lmk_x_14', ' eye_lmk_x_15', ' eye_lmk_x_16', ' eye_lmk_x_17', ' eye_lmk_x_18', ' eye_lmk_x_19', ' eye_lmk_x_20', ' eye_lmk_x_21', ' eye_lmk_x_22', ' eye_lmk_x_23', ' eye_lmk_x_24', ' eye_lmk_x_25', ' eye_lmk_x_26', ' eye_lmk_x_27', ' eye_lmk_x_28', ' eye_lmk_x_29', ' eye_lmk_x_30', ' eye_lmk_x_31', ' eye_lmk_x_32', ' eye_lmk_x_33', ' eye_lmk_x_34', ' eye_lmk_x_35', ' eye_lmk_x_36', ' eye_lmk_x_37', ' eye_lmk_x_38', ' eye_lmk_x_39', ' eye_lmk_x_40', ' eye_lmk_x_41', ' eye_lmk_x_42', ' eye_lmk_x_43', ' eye_lmk_x_44', ' eye_lmk_x_45', ' eye_lmk_x_46', ' eye_lmk_x_47', ' eye_lmk_x_48', ' e

In [7]:
column_names = df_100_dropped.columns.tolist()
print(column_names)
au_column_names = [col for col in column_names if 'AU' in col]
au_r_column_names = [col for col in au_column_names if 'r' in col]
print(len(column_names))

print(df_100_dropped.shape)

['frame', ' face_id', ' confidence', ' success', ' AU01_r', ' AU02_r', ' AU04_r', ' AU05_r', ' AU06_r', ' AU07_r', ' AU09_r', ' AU10_r', ' AU12_r', ' AU14_r', ' AU15_r', ' AU17_r', ' AU20_r', ' AU23_r', ' AU25_r', ' AU26_r', ' AU45_r']
21
(24650, 21)


In [8]:
display(df_100_dropped)

frame   face_id   confidence   success   AU01_r   AU02_r   AU04_r  \
0          1         0         0.03         1      0.0      0.0      0.0   
1          2         0         0.03         0      0.0      0.0      0.0   
2          3         0         0.03         0      0.0      0.0      0.0   
3          4         0         0.03         0      0.0      0.0      0.0   
4          5         0         0.03         0      0.0      0.0      0.0   
...      ...       ...          ...       ...      ...      ...      ...   
24645  24646         0         0.98         1      0.0      0.0      0.0   
24646  24647         0         0.98         1      0.0      0.0      0.0   
24647  24648         0         0.98         1      0.0      0.0      0.0   
24648  24649         0         0.98         1      0.0      0.0      0.0   
24649  24650         0         0.98         1      0.0      0.0      0.0   

        AU05_r   AU06_r   AU07_r  ...   AU10_r   AU12_r   AU14_r   AU15_r  \
0         0.00     0.76     1.37  ...     0.78     0.47     0.86      0.0   
1         0.00     0.25     0.46  ...     0.26     0.16     0.29      0.0   
2         0.00     0.00     0.00  ...     0.00     0.00     0.00      0.0   
3         0.00     0.00     0.00  ...     0.00     0.00     0.00      0.0   
4         0.00     0.00     0.00  ...     0.00     0.00     0.00      0.0   
...        ...      ...      ...  ...      ...      ...      ...      ...   
24645     0.42     2.12     2.01  ...     2.28     1.19     0.64      0.0   
24646     0.42     2.18     2.18  ...     2.30     1.32     0.80      0.0   
24647     0.21     2.20     2.24  ...     2.27     1.43     0.80      0.0   
24648     0.12     2.23     2.22  ...     2.21     1.45     0.71      0.0   
24649     0.00     2.28     2.09  ...     2.20     1.41     0.57      0.0   

        AU17_r   AU20_r   AU23_r   AU25_r   AU26_r   AU45_r  
0         3.40     0.00     1.35     0.29     1.95     0.13  
1         1.13     0.00     0.45     0.10     0.65     0.04  
2         0.00     0.00     0.00     0.00     0.00     0.00  
3         0.00     0.00     0.00     0.00     0.00     0.00  
4         0.00     0.00     0.00     0.00     0.00     0.00  
...        ...      ...      ...      ...      ...      ...  
24645     0.68     0.20     0.00     3.75     2.29     0.00  
24646     0.56     0.20     0.00     3.80     2.22     0.00  
24647     0.42     0.28     0.00     3.73     2.18     0.00  
24648     0.37     0.24     0.00     3.73     2.07     0.00  
24649     0.44     0.38     0.00     3.68     2.07     0.00  

[24650 rows x 21 columns]

In [9]:
# Let's look at a different file in the folder 
csv_file_name_100_7 = "full donation data/100_07.csv"
df_100_7 = extract_and_open_csv(zip_file_path, csv_file_name_100_7)
display(df_100_7)

n_rows, n_columns = df_100_7.shape
print("Number of rows:", n_rows)
print("Number of columns:", n_columns)

frame   face_id   timestamp   confidence   success   gaze_0_x  \
0         1         0        0.00         0.93         1   0.117897   
1         2         0        0.04         0.98         1   0.097296   
2         3         0        0.08         0.98         1   0.118174   
3         4         0        0.12         0.98         1   0.207500   
4         5         0        0.16         0.98         1   0.160233   
...     ...       ...         ...          ...       ...        ...   
2220   2221         0       88.80         0.93         1   0.013683   
2221   2222         0       88.84         0.93         1   0.022169   
2222   2223         0       88.88         0.93         1   0.026135   
2223   2224         0       88.92         0.93         1   0.019407   
2224   2225         0       88.96         0.93         1   0.022808   

       gaze_0_y   gaze_0_z   gaze_1_x   gaze_1_y  ...   AU12_c   AU14_c  \
0      0.100387  -0.987939  -0.036687   0.211267  ...      0.0      1.0   
1      0.113085  -0.988810  -0.061847   0.197766  ...      0.0      1.0   
2      0.152051  -0.981283  -0.035201   0.168773  ...      0.0      1.0   
3      0.259758  -0.943117  -0.147862   0.241424  ...      0.0      1.0   
4      0.211923  -0.964061  -0.073592   0.232193  ...      0.0      1.0   
...         ...        ...        ...        ...  ...      ...      ...   
2220   0.090472  -0.995805  -0.056209   0.162087  ...      0.0      0.0   
2221   0.089485  -0.995741  -0.057457   0.161192  ...      0.0      0.0   
2222   0.094709  -0.995162  -0.075024   0.177604  ...      0.0      0.0   
2223   0.093282  -0.995451  -0.062071   0.163414  ...      0.0      0.0   
2224   0.095643  -0.995154  -0.055603   0.150291  ...      0.0      0.0   

       AU15_c   AU17_c   AU20_c   AU23_c   AU25_c   AU26_c   AU28_c   AU45_c  
0         0.0      1.0      0.0      0.0      0.0      0.0      0.0      0.0  
1         0.0      1.0      0.0      0.0      0.0      0.0      0.0      0.0  
2         1.0      1.0      1.0      0.0      0.0      0.0      0.0      1.0  
3         1.0      1.0      1.0      0.0      0.0      0.0      0.0      1.0  
4         1.0      1.0      1.0      0.0      0.0      0.0      0.0      1.0  
...       ...      ...      ...      ...      ...      ...      ...      ...  
2220      0.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0  
2221      0.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0  
2222      0.0      1.0      1.0      0.0      0.0      0.0      0.0      0.0  
2223      0.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0  
2224      0.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0  

[2225 rows x 714 columns]

Number of rows: 2225
Number of columns: 714


In [10]:
# And another file 
csv_file_name_10_03 = "full donation data/10-03-04-05-06.csv"
df_10_03 = extract_and_open_csv(zip_file_path, csv_file_name_10_03)
display(df_10_03)

n_rows, n_columns = df_10_03.shape
print("Number of rows:", n_rows)
print("Number of columns:", n_columns)

df_10_03 = df_10_03.rename(columns={' face_id': 'face_id'})
print(f"Number of unique values in the face ID column is {df_10_03.face_id.nunique()}.")

frame   face_id   timestamp   confidence   success   gaze_0_x  \
0          1         0        0.00         0.00         0   0.556916   
1          2         0        0.04         0.00         0   0.556916   
2          3         0        0.08         0.00         0   0.556916   
3          4         0        0.12         0.00         0   0.556916   
4          5         0        0.16         0.00         0   0.556916   
...      ...       ...         ...          ...       ...        ...   
19645  19646         0      785.80         0.98         1  -0.287861   
19646  19647         0      785.84         0.98         1  -0.273043   
19647  19648         0      785.88         0.98         1  -0.295750   
19648  19649         0      785.92         0.98         1  -0.293645   
19649  19650         0      785.96         0.98         1  -0.291371   

        gaze_0_y   gaze_0_z   gaze_1_x   gaze_1_y  ...   AU12_c   AU14_c  \
0       0.314565  -0.768696   0.556912   0.314564  ...      0.0      0.0   
1       0.314565  -0.768696   0.556912   0.314564  ...      0.0      0.0   
2       0.314565  -0.768696   0.556912   0.314564  ...      0.0      0.0   
3       0.314565  -0.768696   0.556912   0.314564  ...      0.0      0.0   
4       0.314565  -0.768696   0.556912   0.314564  ...      0.0      0.0   
...          ...        ...        ...        ...  ...      ...      ...   
19645   0.085641  -0.953835  -0.510741   0.144649  ...      0.0      0.0   
19646   0.088023  -0.957966  -0.501093   0.132486  ...      0.0      0.0   
19647   0.079022  -0.951991  -0.506784   0.146299  ...      0.0      0.0   
19648   0.089553  -0.951710  -0.495246   0.159458  ...      0.0      0.0   
19649   0.080139  -0.953247  -0.507093   0.147711  ...      0.0      0.0   

        AU15_c   AU17_c   AU20_c   AU23_c   AU25_c   AU26_c   AU28_c   AU45_c  
0          0.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0  
1          0.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0  
2          0.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0  
3          0.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0  
4          0.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0  
...        ...      ...      ...      ...      ...      ...      ...      ...  
19645      0.0      0.0      0.0      0.0      1.0      1.0      0.0      0.0  
19646      0.0      0.0      0.0      0.0      1.0      1.0      0.0      0.0  
19647      0.0      0.0      0.0      0.0      1.0      1.0      0.0      0.0  
19648      0.0      0.0      0.0      0.0      1.0      1.0      0.0      0.0  
19649      0.0      1.0      0.0      0.0      1.0      1.0      0.0      0.0  

[19650 rows x 714 columns]

Number of rows: 19650
Number of columns: 714
Number of unique values in the face ID column is 1.


In [11]:
# And another file 
csv_file_name_121 = "full donation data/121_04,05,06.csv"
df_121 = extract_and_open_csv(zip_file_path, csv_file_name_121)
display(df_121)

n_rows, n_columns = df_121.shape
print("Number of rows:", n_rows)
print("Number of columns:", n_columns)

frame   face_id   timestamp   confidence   success   gaze_0_x  \
0          1         0        0.00         0.03         1   0.566000   
1          2         0        0.04         0.98         1   0.535502   
2          3         0        0.08         0.98         1   0.541221   
3          4         0        0.12         0.98         1   0.528280   
4          5         0        0.16         0.03         0   0.523882   
...      ...       ...         ...          ...       ...        ...   
29395  29396         0     1175.80         0.98         1   0.139325   
29396  29397         0     1175.84         0.98         1   0.150030   
29397  29398         0     1175.88         0.98         1   0.151403   
29398  29399         0     1175.92         0.98         1   0.146656   
29399  29400         0     1175.96         0.98         1   0.135059   

        gaze_0_y   gaze_0_z   gaze_1_x   gaze_1_y  ...   AU12_c   AU14_c  \
0       0.279817  -0.775465   0.130552   0.214184  ...      1.0      1.0   
1       0.300111  -0.789411   0.061489   0.209813  ...      0.0      0.0   
2       0.274887  -0.794680   0.206141   0.172722  ...      1.0      1.0   
3       0.287404  -0.798949   0.125201   0.241133  ...      0.0      0.0   
4       0.260860  -0.810864   0.087555   0.298181  ...      0.0      0.0   
...          ...        ...        ...        ...  ...      ...      ...   
29395  -0.110797  -0.984029   0.008075  -0.120632  ...      0.0      0.0   
29396  -0.100012  -0.983610   0.015499  -0.117715  ...      0.0      0.0   
29397  -0.087964  -0.984550   0.021941  -0.108117  ...      0.0      0.0   
29398  -0.094707  -0.984643   0.018779  -0.104460  ...      0.0      0.0   
29399  -0.086860  -0.987023  -0.015529  -0.094338  ...      0.0      0.0   

        AU15_c   AU17_c   AU20_c   AU23_c   AU25_c   AU26_c   AU28_c   AU45_c  
0          1.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0  
1          1.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0  
2          1.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0  
3          1.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0  
4          0.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0  
...        ...      ...      ...      ...      ...      ...      ...      ...  
29395      0.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0  
29396      0.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0  
29397      0.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0  
29398      0.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0  
29399      0.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0  

[29400 rows x 714 columns]

Number of rows: 29400
Number of columns: 714


In [16]:
import zipfile

def get_file_names(zip_file_path):
    with zipfile.ZipFile(zip_file_path, 'r') as zip_file:
        file_names = zip_file.namelist()
    return file_names


In [21]:
zip_file_path = '/Users/dionnespaltman/Desktop/downloading/full donation data.zip'
file_names = get_file_names(zip_file_path)
print(file_names)

print(len(file_names))

['full donation data/100_04,05,06.csv', 'full donation data/100_07.csv', 'full donation data/10-03-04-05-06.csv', 'full donation data/10-07.csv', 'full donation data/101_04,05,06.csv', 'full donation data/101_07.csv', 'full donation data/102_04,05,06.csv', 'full donation data/102_07.csv', 'full donation data/103_04,05,06.csv', 'full donation data/103_07.csv', 'full donation data/104_04,05,06.csv', 'full donation data/104_7.csv', 'full donation data/105_03.csv', 'full donation data/105_04 donation not completed. No blood flow.csv', 'full donation data/106_04,05,06.csv', 'full donation data/106_07.csv', 'full donation data/107_03.csv', 'full donation data/107_04,05,06.csv', 'full donation data/107_07.csv', 'full donation data/108_04,05,06.csv', 'full donation data/108_07.csv', 'full donation data/109_04,05,06.csv', 'full donation data/109_07.csv', 'full donation data/110_04.csv', 'full donation data/11-03,04,05,06.csv', 'full donation data/11-07.csv', 'full donation data/111_04,05,06.csv

In [22]:
import csv

In [23]:
csv_file_path = 'file_names.csv'

with open(csv_file_path, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(['File Name'])
    [csv_writer.writerow([file_name]) for file_name in zipfile.ZipFile(zip_file_path, 'r').namelist()]

In [36]:
import re

ids = []

for file_name in file_names:
    match = re.search(r"/(\d{1,3})(?:[-_]|$)", file_name)
    if match:
        ids.append(int(match.group(1)))

ids_sorted = sorted(ids)
#print(ids_sorted)

ids_no_duplicates = set(ids_sorted)
print(ids_no_duplicates)

print(len(ids_no_duplicates))


{5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 138, 139, 140, 141, 142, 143, 144, 145, 146, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332}
185


So in total there are 185 people who participated